In [1]:
from fuvtds_base_class import FUVTDSBase
from fuvtds_base_class import FUVTDSMonitor

import numpy as np

In [2]:
x = FUVTDSBase(PIDs='fuvtds_analysis_list.dat')

  0%|          | 0/1766 [00:00<?, ?it/s]

+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv20eyq_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv22wxq_x1d.fits.gz
+++ Scaling LP3 to LP4 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv22wxq_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv20eyq_x1d.fits.gz
747     0.0
755     0.0
765     0.0
776     0.0
789     0.0
794     0.0
807     0.0
813     0.0
826     0.0
835     0.0
849     0.0
855     0.0
864     0.0
877     0.0
888     0.0
900     0.0
911     0.0
917     0.0
922     0.0
936     0.0
949     0.0
960     0.0
975     0.0
981     0.0
995     0.0
1002    0.0
1017    0.0
1023    0.0
1026    0.0
1040    0.0
1047    0.0
1066    0.0
1466    0.0
1484    0.0
1501    0.0
1510    0.0
1540    0.0
1547    0.0
1576    0.0
1588    0.0
1605    0.0
1624    0.0
1640    0.0
1648    0.0
1662    0.0
1682    0.0
1712    0.0
Name: date-obs, dtype: float64
+++ Scaling LP3 to LP2 using data from datasets: /grp/hst/cos2/cosmo/13967/lcqz03osq_x1d.fits.gz /grp/hst/cos2/cosm

In [3]:
net = np.array([[0.2, 0.3], [0.4, 0.5], [0.6, 0.7]])

In [4]:
net[1]

array([0.4, 0.5])

In [6]:
sum(net)

array([1.2, 1.5])